In [2]:
import os
import pydicom
import nibabel as nib
import numpy as np
from glob import glob
from tqdm import tqdm

In [3]:

def dicom_to_nifti(dicom_dir, output_path, reorient=True):
    """
    Convert a directory of DICOM files to a single NIfTI file.
    
    Parameters:
        dicom_dir (str): Path to directory containing DICOM files
        output_path (str): Path for output NIfTI file (.nii or .nii.gz)
        reorient (bool): Whether to reorient image to standard RAS+ orientation
    """
    # Get all DICOM files in directory
    dicom_files = sorted(glob(os.path.join(dicom_dir, "*")))
    dicom_files = [f for f in dicom_files if not os.path.isdir(f)]
    
    if not dicom_files:
        raise ValueError(f"No DICOM files found in {dicom_dir}")
    
    # Read DICOM files
    dicoms = [pydicom.dcmread(f) for f in tqdm(dicom_files, desc="Reading DICOMs")]
    
    # Skip non-image DICOMs (like RTSTRUCT)
    dicoms = [d for d in dicoms if hasattr(d, 'pixel_array')]
    
    # Sort by instance number (or slice position if instance number missing)
    try:
        dicoms.sort(key=lambda x: int(x.InstanceNumber))
    except:
        dicoms.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    
    # Get pixel data and metadata
    pixel_arrays = [d.pixel_array for d in dicoms]
    affine = np.eye(4)
    
    # Create 3D volume
    volume = np.stack(pixel_arrays, axis=-1)
    
    # Get voxel dimensions from DICOM headers
    try:
        pixel_spacing = dicoms[0].PixelSpacing
        slice_thickness = dicoms[0].SliceThickness
        affine[0, 0] = float(pixel_spacing[0])
        affine[1, 1] = float(pixel_spacing[1])
        affine[2, 2] = float(slice_thickness)
    except:
        print("Warning: Could not extract voxel dimensions from DICOM headers")
    
    # Create NIfTI image
    nifti_img = nib.Nifti1Image(volume, affine)
    
    # Reorient to standard RAS+ if requested
    if reorient:
        nifti_img = nib.as_closest_canonical(nifti_img)
    
    # Save NIfTI file
    nib.save(nifti_img, output_path)
    print(f"Successfully saved NIfTI file to {output_path}")
    

In [4]:
# Example usage
dicom_directory = "dataset/mri-dicom/"  # Replace with your DICOM directory
output_nifti = "dataset/mri-niftii/AdniNiftii.nii.gz"           # .nii or .nii.gz
    
dicom_to_nifti(dicom_directory, output_nifti)

Reading DICOMs: 100%|██████████| 482/482 [00:01<00:00, 381.04it/s]


Successfully saved NIfTI file to dataset/mri-niftii/AdniNiftii.nii.gz
